In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy
import rawpy
import imageio
from PIL import Image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter
from scipy.interpolate import RegularGridInterpolator
import random
from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl

def generate_natural_mountain(seed=None, grid_size=120, save_files=True, interactive=True):
    """
    Generate a natural-looking isolated mountain inspired by Fitz Roy,
    with asymmetric features and realistic terrain characteristics.
    
    Parameters:
    -----------
    seed : int or None
        Random seed for reproducibility
    grid_size : int
        Resolution of the base grid
    save_files : bool
        Whether to save output files
    interactive : bool
        Whether to make plots interactive
    
    Returns:
    --------
    height_map : numpy.ndarray
        2D array representing the height map
    fig_3d : matplotlib.figure.Figure
        Figure object containing the 3D visualization
    fig_2d : matplotlib.figure.Figure
        Figure object containing the 2D height map
    """
    # Set random seed if provided
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    # Create base grid
    x = np.linspace(-3, 3, grid_size)
    y = np.linspace(-3, 3, grid_size)
    xx, yy = np.meshgrid(x, y)
    
    # Initialize height map
    height_map = np.zeros((grid_size, grid_size))
    
    # Create naturalistic base terrain
    base_freq1 = np.random.uniform(1.0, 2.0)
    base_freq2 = np.random.uniform(2.0, 3.0)
    phase1 = np.random.uniform(0, 2*np.pi)
    phase2 = np.random.uniform(0, 2*np.pi)
    
    base_terrain = 0.05 * np.sin(xx * base_freq1 + phase1) * np.cos(yy * base_freq2 + phase2)
    base_terrain += 0.03 * np.sin(xx * base_freq2 * 1.5 + yy * base_freq1 + phase2)
    
    # Add random noise
    noise = 0.02 * np.random.rand(grid_size, grid_size)
    noise = gaussian_filter(noise, sigma=1.5)
    base_terrain += noise
    
    # Add to height map
    height_map += np.maximum(0, base_terrain)
    
    # --------------------------------------------------------
    # Generate the main mountain mass with intentional asymmetry
    # --------------------------------------------------------
    
    # Create an off-center mountain base
    center_x = np.random.uniform(-0.3, 0.3)
    center_y = np.random.uniform(-0.3, 0.3)
    
    # Create asymmetric distance field
    x_scale = np.random.uniform(0.8, 1.2)
    y_scale = np.random.uniform(0.8, 1.2)
    r = np.sqrt(((xx - center_x) * x_scale)**2 + ((yy - center_y) * y_scale)**2)
    
    # Add directional warping
    warp_strength = np.random.uniform(0.1, 0.2)
    warp_dir_x = np.random.uniform(-1, 1)
    warp_dir_y = np.random.uniform(-1, 1)
    r += warp_strength * (warp_dir_x * xx + warp_dir_y * yy)
    
    # Create the main mountain mass with complex falloff
    base_height = np.random.uniform(0.8, 1.0)
    falloff = np.random.uniform(0.4, 0.6)
    mountain_base = base_height * np.exp(-r**2 / falloff)
    
    # Add asymmetric distortion
    distortion = np.zeros_like(mountain_base)
    for _ in range(3):
        freq_x = np.random.uniform(1.5, 3.0)
        freq_y = np.random.uniform(1.5, 3.0)
        phase = np.random.uniform(0, 2*np.pi)
        amp = np.random.uniform(0.1, 0.2)
        distortion += amp * np.sin(xx * freq_x + yy * freq_y + phase)
    
    # Apply distortion
    distortion *= mountain_base
    mountain_base += distortion
    
    # Add to height map
    height_map = np.maximum(height_map, mountain_base)
    
    # --------------------------------------------------------
    # Generate Fitz Roy's characteristic jagged ridges and peaks
    # --------------------------------------------------------
    
    # Find starting point for the ridge
    highest_point = np.unravel_index(np.argmax(mountain_base), mountain_base.shape)
    high_y, high_x = highest_point
    start_x = x[high_x] + np.random.uniform(-0.5, 0.5)
    start_y = y[high_y] + np.random.uniform(-0.5, 0.5)
    
    # Generate a natural ridge path
    ridge_length = np.random.randint(4, 8)
    ridge_x = [start_x]
    ridge_y = [start_y]
    
    # Starting direction
    direction = np.random.uniform(0, 2*np.pi)
    
    # Create the ridge by "walking" in slightly random directions
    for _ in range(ridge_length):
        # Slightly change direction
        direction += np.random.uniform(-0.5, 0.5)
        
        # Calculate next point
        step_length = np.random.uniform(0.3, 0.7)
        next_x = ridge_x[-1] + step_length * np.cos(direction)
        next_y = ridge_y[-1] + step_length * np.sin(direction)
        
        # Ensure we stay in bounds
        if -2.5 < next_x < 2.5 and -2.5 < next_y < 2.5:
            ridge_x.append(next_x)
            ridge_y.append(next_y)
    
    # Add jagged peaks along the ridge
    num_peaks = len(ridge_x)
    
    # Randomize peak heights with a general trend
    position_factor = np.linspace(0, 1, num_peaks)
    position_factor = 4 * position_factor * (1 - position_factor)
    peak_heights = 0.6 + 0.4 * position_factor + 0.2 * np.random.rand(num_peaks)
    
    # Ensure one main peak (Fitz Roy's characteristic spire)
    main_peak_idx = np.random.randint(num_peaks // 3, 2 * num_peaks // 3)
    peak_heights[main_peak_idx] = 1.0
    
    # Add the peaks with varying characteristics
    for i in range(num_peaks):
        peak_x, peak_y = ridge_x[i], ridge_y[i]
        peak_height = peak_heights[i]
        
        # Vary the sharpness and shape of each peak
        sharpness = np.random.uniform(0.05, 0.15)
        if i == main_peak_idx:
            sharpness *= 0.7  # Make the main peak especially sharp
        
        # Asymmetric falloff in different directions
        asymm_x = np.random.uniform(0.7, 1.3)
        asymm_y = np.random.uniform(0.7, 1.3)
        rotation = np.random.uniform(0, 2*np.pi)
        
        # Create rotated, asymmetric distance field
        dx = xx - peak_x
        dy = yy - peak_y
        dx_rot = dx * np.cos(rotation) - dy * np.sin(rotation)
        dy_rot = dx * np.sin(rotation) + dy * np.cos(rotation)
        r_asym = np.sqrt((dx_rot * asymm_x)**2 + (dy_rot * asymm_y)**2)
        
        # Create peak with variable power
        power = np.random.uniform(2.0, 3.5)
        if i == main_peak_idx:
            power = 3.5  # Very sharp main peak
            
        peak = peak_height * np.exp(-r_asym**2 / sharpness)
        peak = peak ** power
        
        # Add to height map
        height_map = np.maximum(height_map, peak)
    
    # --------------------------------------------------------
    # Add secondary ridges
    # --------------------------------------------------------
    
    num_secondary = np.random.randint(2, 5)
    for _ in range(num_secondary):
        # Start from a random point on the main ridge
        idx = np.random.randint(0, len(ridge_x))
        sec_start_x = ridge_x[idx]
        sec_start_y = ridge_y[idx]
        
        # Determine ridge direction
        if idx < len(ridge_x) - 1:
            ridge_dir = np.arctan2(ridge_y[idx+1] - ridge_y[idx], ridge_x[idx+1] - ridge_x[idx])
        else:
            ridge_dir = np.arctan2(ridge_y[idx] - ridge_y[idx-1], ridge_x[idx] - ridge_x[idx-1])
            
        # Perpendicular direction with randomness
        sec_dir = ridge_dir + np.pi/2 + np.random.uniform(-0.5, 0.5)
        
        # Create secondary ridge
        sec_length = np.random.randint(2, 4)
        sec_x = [sec_start_x]
        sec_y = [sec_start_y]
        
        for _ in range(sec_length):
            # Add meandering
            sec_dir += np.random.uniform(-0.4, 0.4)
            
            # Calculate next point
            step_length = np.random.uniform(0.2, 0.5)
            next_x = sec_x[-1] + step_length * np.cos(sec_dir)
            next_y = sec_y[-1] + step_length * np.sin(sec_dir)
            
            # Stay in bounds
            if -2.5 < next_x < 2.5 and -2.5 < next_y < 2.5:
                sec_x.append(next_x)
                sec_y.append(next_y)
        
        # Decrease height away from main ridge
        sec_heights = 0.7 * peak_heights[idx] * np.linspace(0.9, 0.3, len(sec_x))
        sec_heights += 0.1 * np.random.rand(len(sec_x))
        
        # Add peaks for secondary ridge
        for i in range(len(sec_x)):
            peak_x, peak_y = sec_x[i], sec_y[i]
            peak_height = sec_heights[i]
            
            sharpness = np.random.uniform(0.05, 0.12)
            r = np.sqrt((xx - peak_x)**2 + (yy - peak_y)**2)
            
            # Add asymmetry
            asymm = 0.2 * np.sin(xx * 2 + yy * 2)
            r_mod = r + asymm * np.clip(0.3 - r, 0, 0.3) * 3
            
            peak = peak_height * np.exp(-r_mod**2 / sharpness)
            peak = peak ** np.random.uniform(1.8, 2.5)
            
            # Add to height map
            height_map = np.maximum(height_map, peak)
    
    # --------------------------------------------------------
    # Add surface details and texture
    # --------------------------------------------------------
    
    # Multiple texture scales
    large_scale = 0.07 * np.random.rand(grid_size, grid_size)
    large_scale = gaussian_filter(large_scale, sigma=1.0)
    
    medium_scale = 0.04 * np.random.rand(grid_size, grid_size)
    medium_scale = gaussian_filter(medium_scale, sigma=0.5)
    
    small_scale = 0.02 * np.random.rand(grid_size, grid_size)
    small_scale = gaussian_filter(small_scale, sigma=0.2)
    
    # Combine textures
    height_factor = np.clip(height_map / 0.2, 0, 1)
    texture = large_scale + medium_scale * height_factor + small_scale * height_factor**2
    
    # Add texture preserving peaks
    height_map_with_texture = height_map + texture * np.clip(0.7 - height_map, 0, 0.7)
    
    # Add erosion features
    gradient_x, gradient_y = np.gradient(height_map)
    slope = np.sqrt(gradient_x**2 + gradient_y**2)
    
    erosion_mask = (slope > 0.1) & (height_map > 0.1)
    erosion = 0.05 * np.random.rand(grid_size, grid_size) * erosion_mask
    erosion = gaussian_filter(erosion, sigma=0.1)
    
    # Final height map
    height_map_final = height_map_with_texture - erosion
    height_map_final = np.maximum(0, height_map_final)
    
    # --------------------------------------------------------
    # Create 2D height map visualization (more detailed)
    # --------------------------------------------------------
    
    # Create custom terrain colormap
    terrain_colors = {
        0.0: (0.0, 0.4, 0.0),  # Dark green (base)
        0.3: (0.2, 0.6, 0.2),  # Green vegetation
        0.45: (0.6, 0.5, 0.3),  # Brown/tan (lower slopes)
        0.6: (0.7, 0.65, 0.6),  # Light gray (rocky)
        0.75: (0.8, 0.8, 0.8),  # Light gray/white (approaching snow)
        1.0: (1.0, 1.0, 1.0)    # White (snow-capped peaks)
    }
    
    # Create the colormap
    cmap_terrain = LinearSegmentedColormap.from_list(
        'terrain_natural', 
        [(pos, color) for pos, color in terrain_colors.items()],
        N=256
    )
    
    # Create figure for 2D visualization
    fig_2d = plt.figure(figsize=(12, 10))
    ax_2d = fig_2d.add_subplot(111)
    
    # Calculate hillshade for 3D-like effect
    dx, dy = np.gradient(height_map_final)
    slope = np.pi/2 - np.arctan(np.sqrt(dx**2 + dy**2))
    aspect = np.arctan2(-dy, dx)
    
    # Light direction
    altitude = np.pi/4  # Light altitude (radians)
    azimuth = np.pi/2    # Light direction (radians)
    
    # Calculate shaded relief
    shaded = np.sin(altitude) * np.sin(slope) + np.cos(altitude) * np.cos(slope) * np.cos(azimuth - aspect)
    
    # Plot with combined coloring
    im = ax_2d.imshow(height_map_final, cmap=cmap_terrain, extent=[-3, 3, -3, 3])
    ax_2d.imshow(shaded, cmap='gray', alpha=0.3, extent=[-3, 3, -3, 3])
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax_2d)
    cbar.set_label('Elevation (km)', fontsize=12)
    
    # Add title and labels
    ax_2d.set_title('Mountain Height Map', fontsize=16)
    ax_2d.set_xlabel('X (km)', fontsize=12)
    ax_2d.set_ylabel('Y (km)', fontsize=12)
    
    # Add grid lines for reference
    ax_2d.grid(color='gray', linestyle='-', linewidth=0.3, alpha=0.5)
    
    # Save image if requested
    if save_files:
        plt.savefig('mountain_height_map.png', dpi=300, bbox_inches='tight')
    
    # --------------------------------------------------------
    # Create 3D point cloud / histogram visualization
    # --------------------------------------------------------
    
    # Convert the height map to 3D points
    points = []
    for i in range(grid_size):
        for j in range(grid_size):
            if height_map_final[i, j] > 0.03:
                # Number of points proportional to height
                height = height_map_final[i, j]
                num_points = int(height * 300)
                
                for _ in range(num_points):
                    # Add small random jitter to positions
                    x_pos = x[j] + np.random.normal(0, 0.03)
                    y_pos = y[i] + np.random.normal(0, 0.03)
                    
                    # Height with small variations
                    z_pos = height * 3.0 * (1 + 0.05 * np.random.randn())
                    
                    points.append([x_pos, y_pos, z_pos])
    
    # Convert to numpy array
    point_data = np.array(points)
    
    # Create histogram
    bins_x = 30
    bins_y = 30
    bins_z = 50  # Higher resolution for height
    hist, edges = np.histogramdd(point_data, bins=[bins_x, bins_y, bins_z])
    
    # Get bin centers
    x_centers = edges[0][:-1] + (edges[0][1] - edges[0][0]) / 2
    y_centers = edges[1][:-1] + (edges[1][1] - edges[1][0]) / 2
    z_centers = edges[2][:-1] + (edges[2][1] - edges[2][0]) / 2
    
    # Create high-resolution grid
    sub_divisions = 3
    x_fine = np.linspace(edges[0][0], edges[0][-1], bins_x * sub_divisions)
    y_fine = np.linspace(edges[1][0], edges[1][-1], bins_y * sub_divisions)
    z_fine = np.linspace(edges[2][0], edges[2][-1], bins_z * sub_divisions)
    
    # Set up interpolation
    interp_func = RegularGridInterpolator((x_centers, y_centers, z_centers),
                                          hist,
                                          bounds_error=False,
                                          fill_value=0)
    
    # Prepare coordinate arrays
    x_mesh, y_mesh, z_mesh = np.meshgrid(x_fine, y_fine, z_fine, indexing='ij')
    interp_points = np.vstack([x_mesh.flatten(), y_mesh.flatten(), z_mesh.flatten()]).T
    
    # Interpolate values
    high_res_values = interp_func(interp_points)
    
    # Add variations for texture
    variations = 1 + 0.04 * np.sin(interp_points[:, 0] * 5) * np.cos(interp_points[:, 1] * 7)
    high_res_values = high_res_values * variations
    
    # Reshape to grid
    high_res_hist = high_res_values.reshape(len(x_fine), len(y_fine), len(z_fine))
    
    # Extract coordinates for voxels above threshold
    x_pos, y_pos, z_pos, heights = [], [], [], []
    threshold = 0.005
    
    for i in range(len(x_fine)):
        for j in range(len(y_fine)):
            for k in range(len(z_fine)):
                if high_res_hist[i, j, k] > threshold:
                    x_pos.append(x_fine[i])
                    y_pos.append(y_fine[j])
                    z_pos.append(z_fine[k])
                    heights.append(high_res_hist[i, j, k])
    
    # Convert to numpy arrays
    x_pos = np.array(x_pos)
    y_pos = np.array(y_pos)
    z_pos = np.array(z_pos)
    heights = np.array(heights)
    
    # Save data for future use
    if save_files:
        np.save("mountain_height_map.npy", height_map_final)
        np.save("mountain_point_data.npy", point_data)
    
    # -------------------------------------------------
    # Create interactive 3D visualization
    # -------------------------------------------------
    
    # Create figure for 3D visualization
    fig_3d = plt.figure(figsize=(14, 12), facecolor='white')
    ax_3d = fig_3d.add_subplot(111, projection='3d')
    ax_3d.set_facecolor('white')
    
    # Set bar dimensions
    dx = (x_fine[1] - x_fine[0]) * 0.95
    dy = (y_fine[1] - y_fine[0]) * 0.95
    
    # Create natural color gradient
    normalized_heights = heights / np.max(heights)
    color_array = np.zeros((len(heights), 4))
    
    for i in range(len(heights)):
        h = normalized_heights[i]
        
        # Create a natural mountain color gradient
        if h < 0.25:
            r, g, b = 0.2, 0.4 + 0.2 * h, 0.1
        elif h < 0.7:
            factor = (h - 0.25) / 0.45
            r = 0.4 + 0.3 * factor
            g = 0.3 + 0.3 * factor
            b = 0.3 + 0.4 * factor
        else:
            factor = (h - 0.7) / 0.3
            r = 0.7 + 0.3 * factor
            g = 0.7 + 0.3 * factor
            b = 0.8 + 0.2 * factor
        
        # Add slight noise
        noise = np.random.uniform(-0.05, 0.05)
        r = np.clip(r + noise, 0, 1)
        g = np.clip(g + noise, 0, 1)
        b = np.clip(b + noise, 0, 1)
        
        # Opacity increases with height
        alpha = 0.3 + 0.7 * h
        color_array[i] = [r, g, b, alpha]
    
    # Plot bars
    bars = ax_3d.bar3d(x_pos, y_pos, z_pos, dx, dy, heights * 0.2, color=color_array, shade=True)
    
    # Adjust viewing angle
    ax_3d.view_init(30, 45)
    ax_3d.grid(False)
    
    # Set aspect ratio
    ax_3d.set_box_aspect([1, 1, 0.6])
    
    # Set labels
    ax_3d.set_xlabel('X (km)', fontsize=12)
    ax_3d.set_ylabel('Y (km)', fontsize=12)
    ax_3d.set_zlabel('Elevation (km)', fontsize=12)
    
    # Set title
    ax_3d.set_title('Interactive 3D Mountain Model', fontsize=16)
    
    # Add information box
    ax_3d.text2D(0.02, 0.98,
                f"Natural Mountain\n"
                f"Points: {len(point_data):,}\n"
                f"Voxels: {len(heights):,}\n"
                f"Peak: {np.max(z_pos):.2f} km",
                transform=ax_3d.transAxes,
                fontsize=10,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Save 3D visualization
    if save_files:
        plt.savefig('mountain_3d_model.png', dpi=300, bbox_inches='tight')
    
    # Make the 3D plot interactive if requested
    if interactive:
        plt.tight_layout()
        
        # Add instruction text for interaction
        instruction_text = ax_3d.text2D(0.5, 0.02,
                         "Click and drag to rotate. Scroll to zoom.",
                         transform=ax_3d.transAxes,
                         fontsize=10,
                         horizontalalignment='center',
                         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    return height_map_final, fig_3d, fig_2d

# Function to create a standalone interactive viewer
def create_interactive_viewer(height_map, title="Interactive Mountain Viewer"):
    """
    Create a standalone interactive viewer for the mountain.
    
    Parameters:
    -----------
    height_map : numpy.ndarray
        2D height map array
    title : str
        Window title
    
    Returns:
    --------
    fig : matplotlib.figure.Figure
        Interactive figure
    """
    from matplotlib.widgets import Slider
    
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract dimensions
    grid_size = height_map.shape[0]
    x = np.linspace(-3, 3, grid_size)
    y = np.linspace(-3, 3, grid_size)
    X, Y = np.meshgrid(x, y)
    
    # Initial view
    elev, azim = 30, 45
    
    # Plot the surface
    surf = ax.plot_surface(X, Y, height_map * 3.0, cmap='terrain',
                          linewidth=0, antialiased=True, alpha=0.8)
    
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Elevation (km)')
    ax.set_title(title)
    
    # Set initial view
    ax.view_init(elev, azim)
    
    # Add sliders for interactive control
    ax_elev = plt.axes([0.25, 0.02, 0.65, 0.03])
    ax_azim = plt.axes([0.25, 0.06, 0.65, 0.03])
    
    slider_elev = Slider(ax_elev, 'Elevation', 0, 90, valinit=elev)
    slider_azim = Slider(ax_azim, 'Azimuth', 0, 360, valinit=azim)
    
    def update(val):
        ax.view_init(slider_elev.val, slider_azim.val)
        fig.canvas.draw_idle()
    
    slider_elev.on_changed(update)
    slider_azim.on_changed(update)
    
    # Add instruction text
    instruction_text = ax.text2D(0.5, 0.92,
                    "Use sliders to rotate. Click and drag to pan. Scroll to zoom.",
                    transform=ax.transAxes,
                    fontsize=10,
                    horizontalalignment='center',
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    return fig


if __name__ == "__main__":
    # Set a random seed (None for different results each time)
    seed = 42
    
    # Generate the mountain
    height_map, fig_3d, fig_2d = generate_natural_mountain(seed=seed, grid_size=120)
    
    # Create an additional interactive viewer with convenient controls
    interactive_fig = create_interactive_viewer(height_map, "Interactive Mountain Explorer")
    
    # Show all plots
    plt.show()

KeyboardInterrupt: 

: 